<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/%E5%B0%9D%E8%AF%95%E8%BD%AC%E5%8C%96RoleLLM%E4%B8%BAHaruhi%E6%A0%BC%E5%BC%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install transformers openai tiktoken langchain chromadb

In [ ]:
!git clone https://huggingface.co/datasets/ZenMoore/RoleBench

Cloning into 'RoleBench'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (255/255), done.
remote: Compressing objects: 100% (255/255), done.
remote: Total 259 (delta 9), reused 0 (delta 0), pack-reused 4
Receiving objects: 100% (259/259), 20.89 MiB | 6.73 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Filtering content: 100% (8/8), 366.76 MiB | 41.52 MiB/s, done.


- [ ] 分析desc
- [ ] 解析jsonl，数一下token

In [ ]:
desc_path = "/content/RoleBench/profiles-eng/desc.json"
movie_name_path = "/content/RoleBench/profiles-eng/scripts.json"

# 请补全下面的python代码为我解析这两个json文件

import json

with open(desc_path) as f:
    desc = json.load(f)

with open(movie_name_path) as f:
    movie_names = json.load(f)

In [ ]:
role_names = [x for x in desc.keys()]
print(role_names)

['Doctor Who', 'Mary Sibley', 'Lucifer Morningstar', 'Twilight Sparkle', 'Oliver Queen', 'Angel', 'Queen Catherine', 'Dr. Hannibal Lecter', 'HAL 9000', 'Colonel Nathan R. Jessep', 'Antonio Salieri', 'Stifler', 'Paul Vitti', 'Alvy Singer', 'Violet Weston', 'Sheldon Cooper', 'Willie Soke', 'Gaston', 'The Dude', 'Murphy MacManus', 'Paul Conroy', 'Truman Capote', 'Mater', 'Andrew Detmer', 'Coriolanus', 'Benjamin Button', 'John Keating', 'Wade Wilson', 'Jim Morrison', 'Queen Elizabeth I', 'Coach Eric Taylor', 'Jeff Spicoli', 'Fred Flintstone', 'Freddy Krueger', 'Tyrion Lannister', 'James Brown', 'Walt Kowalski', 'John Coffey', 'Theodore Twombly', 'Gregory House', 'Sonny', 'Colonel Hans Landa', 'Judge Dredd', 'Juno MacGuff', 'Po', 'Klaus Mikaelson', 'Professor G.H. Dorr', 'Fletcher Reede', 'Abraham Lincoln', 'Frank T.J. Mackey', 'Malcolm X', 'Leonard Shelby', 'Harvey Milk', 'Randle McMurphy', 'Jack Sparrow', 'John Dillinger', 'Lestat de Lioncourt', 'Tyler Hawkins', 'Caesar', 'Jack', 'Leroy J

In [ ]:
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

In [ ]:
def get_script_path( role_name ):
    return f"/content/RoleBench/profiles-eng/profiles-eng-{role_name}.jsonl"


def read_jsonl(file_path):
    data = []
    with open(file_path, encoding='utf-8') as f:
        for line in f:
            try:
                data.append(json.loads(line))
            except:
                pass
    return data

ALLOW_SPLIT_TOKEN_LEN = 300
ENFORCE_SPLIT_TOKEN_LEN = 700

def divide_data_into_chunk( data ):
    chunk_data = []
    last_act = -1
    last_diag = -1
    current_chunk = ""
    len_current_chunk = 0
    data.append({"act_id":-1, "diag_id":-1, "content":"","role":""})
    for d in data:
        act_id = d['act_id']
        diag_id = d['diag_id']
        role = d['role']

        split_flag = False

        if str.lower(role).startswith('narr'):
            current_content = role + ":" + d['content'] + '\n'
        else:
            current_content = role + ":「" + d['content'] + '」\n'

        len_current_content = len(enc.encode(current_content))

        if len_current_content > ENFORCE_SPLIT_TOKEN_LEN:
            n = len(d['content'])
            while len(enc.encode(d['content'][:n])) > ENFORCE_SPLIT_TOKEN_LEN:
                n -= 20
            current_content = role + ":「" + d['content'] + '」\n'

            len_current_content = len(enc.encode(current_content))

        if act_id != last_act:
            split_flag = True

        if act_id == last_act and diag_id != last_diag:
            if len_current_content + len_current_chunk  > ALLOW_SPLIT_TOKEN_LEN:
                split_flag = True

        if len_current_content + len_current_chunk > ENFORCE_SPLIT_TOKEN_LEN:
            split_flag = True

        if split_flag == True:
            if current_chunk != "":
                chunk_data.append(current_chunk)
            last_act = act_id
            last_diag = diag_id
            current_chunk = current_content
            len_current_chunk = len_current_content
        else:
            current_chunk += current_content
            len_current_chunk += len_current_content

    return chunk_data


    # break


In [ ]:
print(chunk_data[4])

NameError: ignored

In [ ]:
!mkdir /content/output

In [ ]:
import os

for role_name in desc:
    print(role_name)
    script_path = get_script_path(role_name)
    data = read_jsonl(script_path)
    chunk_data = divide_data_into_chunk(data)
    max_len = 0
    for c in chunk_data:
        max_len = max(max_len,len(enc.encode(c)))
    print(role_name , ' max chunk ', max_len , ' n_chunk = ', len(chunk_data))

    # 创建角色名称的文件夹
    output_dir = f"/content/output/{role_name}"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 将chunk写入文件
    for i, chunk in enumerate(chunk_data):
        file_path = f"{output_dir}/{i}.txt"
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(chunk)
    break

In [ ]:
max_token_len = 0

def get_system_prompt( role_name ):
    character = role_name
    series = movie_names[role_name]
    original_prompt = f'''I want you to act like {character} from {series}.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like {character} using the tone, manner and vocabulary {character} would use.
You must know all of the knowledge of {character}.

'''
    original_prompt += desc[role_name]

    return original_prompt



for role_name in desc:
    character = role_name
    series = movie_names[role_name]
    original_prompt = f'''I want you to act like {character} from {series}.
If others‘ questions are related with the novel, please try to reuse the original lines from the novel.
I want you to respond and answer like {character} using the tone, manner and vocabulary {character} would use.
You must know all of the knowledge of {character}.

'''
    original_prompt += desc[role_name]

    current_len = len(enc.encode(original_prompt))

    max_token_len = max(current_len, max_token_len)


print(max_token_len)

285


In [ ]:
# from transformers import AutoTokenizer, AutoModel
# import torch
# # Sentences we want sentence embeddings for
# sentences = ["an apple", chunk_data[1],chunk_data[2],chunk_data[0]]

# # Load model from HuggingFace Hub
# tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-small-en-v1.5')
# model = AutoModel.from_pretrained('BAAI/bge-small-en-v1.5')
# model.eval()

# # Tokenize sentences
# encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt', max_length = 512)
# # for s2p(short query to long passage) retrieval task, add an instruction to query (not add instruction for passages)
# # encoded_input = tokenizer([instruction + q for q in queries], padding=True, truncation=True, return_tensors='pt')

# # Compute token embeddings
# with torch.no_grad():
#     model_output = model(**encoded_input)
#     # Perform pooling. In this case, cls pooling.
#     sentence_embeddings = model_output[0][:, 0]
# # normalize embeddings
# sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
# print(sentence_embeddings.shape)

encoded_input有三个分量

print(encoded_input.input_ids.shape)
print(encoded_input.token_type_ids.shape)
print(encoded_input.attention_mask.shape)

这三个shape是一样的，都是4*617

我希望发现encoded_input.input_ids超过M * 512时

自动把三个tensor都截断为 M * 512

请用python为我实现

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

_bge_model = None
_bge_tokenizer = None

def get_bge_embeddings( sentences ):
    # unsafe ensure batch size by yourself

    global _bge_model
    global _bge_tokenizer

    if _bge_model is None:
        from transformers import AutoTokenizer, AutoModel
        _bge_tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-small-en-v1.5')
        _bge_model = AutoModel.from_pretrained('BAAI/bge-small-en-v1.5')

    _bge_model.eval()

    # Tokenize sentences
    encoded_input = _bge_tokenizer(sentences, padding=True, truncation=True, return_tensors='pt', max_length = 512)

    # Compute token embeddings
    with torch.no_grad():
        model_output = _bge_model(**encoded_input)
        # Perform pooling. In this case, cls pooling.
        sentence_embeddings = model_output[0][:, 0]
    # normalize embeddings
    sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
    return sentence_embeddings.cpu().tolist()

def get_bge_embedding( text_or_texts ):
    if isinstance(text_or_texts, str):
        return get_bge_embeddings([text_or_texts])[0]
    else:
        return get_bge_embeddings_safe(text_or_texts)

bge_batch_size = 16

import math
from tqdm import tqdm

def get_bge_embeddings_safe(sentences):

    embeddings = []

    num_batches = math.ceil(len(sentences) / bge_batch_size)

    for i in tqdm( range(num_batches) ):
        # print("run bge with batch ", i)
        start_index = i * bge_batch_size
        end_index = min(len(sentences), start_index + bge_batch_size)
        batch = sentences[start_index:end_index]
        embs = get_bge_embeddings(batch)
        embeddings.extend(embs)

    return embeddings

test_embed = get_bge_embeddings_safe(chunk_data[:35])
test_single = get_bge_embedding(chunk_data[0])


In [ ]:
print(len(test_embed))
print(test_embed[33])

In [ ]:
%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev

/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 782, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 782 (delta 56), reused 27 (delta 12), pack-reused 693
Receiving objects: 100% (782/782), 105.71 MiB | 36.93 MiB/s, done.
Resolving deltas: 100% (386/386), done.
/content/Haruhi-2-Dev


In [ ]:

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
import os
import openai
key = "sk-WafsA4Ce"
key_bytes = key.encode()
os.environ["OPENAI_API_KEY"] = key_bytes.decode('utf-8')

openai.api_key = key



In [ ]:
# def get_embedding_openai(text, model="text-embedding-ada-002"):
#     text = text.replace("\n", " ")
#     return openai.Embedding.create(input=[text], model=model)['data'][0]['embedding']

# print(get_embedding_openai(chunk_data[0]))

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")

# text = chunk_data[0]
# text = text.replace("\n", " ")
# result = embeddings.embed_query(text)

In [ ]:
print(result)
print(len(result))

NameError: ignored

In [ ]:
from tqdm import tqdm
def get_embeddings_openai( texts ):
    ans = []
    embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
    for text in tqdm(texts):
        text = text.replace("\n", " ")
        try:
            result = embeddings.embed_query(text)
        except:
            print('warning!')
            result = [0] * 1536
        ans.append(result)
    return ans

# embeds = get_embeddings_openai(chunk_data[:3])


In [ ]:
def package_role( system_prompt, texts_path , embedding ):
    datas = []

    # 暂时只有一种embedding 'luotuo_openai'
    embed_name = 'luotuo_openai'

    datas.append({ 'text':system_prompt , embed_name:'system_prompt'})
    datas.append({ 'text':'Reserve Config Setting Here' , embed_name:'config'})


    # debug_count = 3

    # for file in os.listdir(texts_path):

    files = os.listdir(texts_path)

    for i in tqdm.tqdm(range(len(files))):
        file = files[i]
        # if file name end with txt
        if file.endswith(".txt"):
            file_path = os.path.join(texts_path, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                current_str = f.read()
                current_vec = embedding(current_str)
                encode_vec = float_array_to_base64(current_vec)
                datas.append({ 'text':current_str , embed_name:encode_vec})

                # debug_count -= 1
                # if debug_count == 0:
                #     break
    return datas

先处理数据较少的角色

将角色根据chunk数从小到大排列

In [ ]:
# role_infos = []

# for role_name in desc:
#   script_path = get_script_path(role_name)
#   data = read_jsonl(script_path)
#   chunk_data = divide_data_into_chunk(data)

#   max_len = 0
#   for c in chunk_data:
#     max_len = max(max_len,len(enc.encode(c)))

#   len_chunk = len(chunk_data)

#   role_infos.append((role_name, len_chunk))

# role_infos.sort(key=lambda x: x[1])

# sorted_names = [x[0] for x in role_infos]

In [ ]:
print(sorted_names)

['Caesar', 'Sonny', 'Angel', 'Jigsaw', 'John Doe', 'Freddy Krueger', 'Colonel Hans Landa', 'Gregory House', 'Gaston', 'HAL 9000', 'Mark Renton', 'Coriolanus', 'Oliver Queen', 'Bruno Antony', 'D_Artagnan', 'Dr. Frank-N-Furter', 'Tugg Speedman', 'Stifler', 'Jeff Spicoli', 'Rorschach', 'Paul Vitti', 'Logan', 'Judge Dredd', 'Karl Childers', 'Rachel Lang', 'Queen Elizabeth I', 'Tyrion Lannister', 'John Keating', 'Wade Wilson', 'Lyn Cassady', 'Dr. Hannibal Lecter', 'Violet Weston', 'Po', 'Malcolm X', 'Willie Soke', 'Jack Torrance', 'Alvy Singer', 'Colonel Nathan R. Jessep', 'Andrew Detmer', 'Fred Flintstone', 'Frank T.J. Mackey', 'Stephen Hawking', 'Lestat de Lioncourt', 'Jack Sparrow', 'John Coffey', 'Murphy MacManus', 'John Dillinger', 'Jackie Moon', 'Peter Parker', 'Abraham Lincoln', 'James Carter', 'Tyler Hawkins', 'Stanley Ipkiss', 'Mater', 'Professor G.H. Dorr', 'Juno MacGuff', 'Seth', 'Sherlock Holmes', 'Truman Capote', 'Shrek', 'Travis Bickle', 'Jack', 'Tom Ripley', 'The Dude', 'Davi

In [ ]:
sorted_names = ['Caesar', 'Sonny', 'Angel', 'Jigsaw', 'John Doe', 'Freddy Krueger', 'Colonel Hans Landa', 'Gregory House', 'Gaston', 'HAL 9000', 'Mark Renton', 'Coriolanus', 'Oliver Queen', 'Bruno Antony', 'D_Artagnan', 'Dr. Frank-N-Furter', 'Tugg Speedman', 'Stifler', 'Jeff Spicoli', 'Rorschach', 'Paul Vitti', 'Logan', 'Judge Dredd', 'Karl Childers', 'Rachel Lang', 'Queen Elizabeth I', 'Tyrion Lannister', 'John Keating', 'Wade Wilson', 'Lyn Cassady', 'Dr. Hannibal Lecter', 'Violet Weston', 'Po', 'Malcolm X', 'Willie Soke', 'Jack Torrance', 'Alvy Singer', 'Colonel Nathan R. Jessep', 'Andrew Detmer', 'Fred Flintstone', 'Frank T.J. Mackey', 'Stephen Hawking', 'Lestat de Lioncourt', 'Jack Sparrow', 'John Coffey', 'Murphy MacManus', 'John Dillinger', 'Jackie Moon', 'Peter Parker', 'Abraham Lincoln', 'James Carter', 'Tyler Hawkins', 'Stanley Ipkiss', 'Mater', 'Professor G.H. Dorr', 'Juno MacGuff', 'Seth', 'Sherlock Holmes', 'Truman Capote', 'Shrek', 'Travis Bickle', 'Jack', 'Tom Ripley', 'The Dude', 'David Aames', 'Twilight Sparkle', 'Antonio Salieri', 'Judy Hoops', 'Randle McMurphy', 'Thor', 'Walt Kowalski', 'Fletcher Reede', 'Theodore Twombly', 'James Brown', 'Paul Conroy', 'James Bond', 'Queen Catherine', 'Harvey Milk', 'Caden Cotard', 'Leonard Shelby', 'Jim Morrison', 'Pat Solitano', 'Benjamin Button', 'Robert Angier', 'Lucifer Morningstar', 'Jordan Belfort', 'Coach Eric Taylor', 'Mary Sibley', 'Klaus Mikaelson', 'Raylan Givens', 'Sheldon Cooper', 'Michael Scott', 'Leroy Jethro Gibbs', 'Doctor Who', 'Blair Waldorf']

In [ ]:
# for role_name in sorted_names:
#     script_path = get_script_path(role_name)
#     data = read_jsonl(script_path)
#     chunk_data = divide_data_into_chunk(data)
#     print(len(chunk_data))
#     break

13


In [ ]:
from ChatHaruhi.utils import get_bge_embeddings_safe

# embed_openai = get_embeddings_openai(chunk_data[:])
# embed_bge = get_bge_embeddings_safe(chunk_data[:])


In [ ]:
# datas = []

# system_prompt = get_system_prompt( role_name )

# embed_name = 'luotuo_openai'
# embed_name_bge = 'bge_en_s15'

# datas.append({ 'text':system_prompt , embed_name:'system_prompt', embed_name_bge:'system_prompt'})
# datas.append({ 'text':'Reserve Config Setting Here' , embed_name:'config', embed_name_bge:'config'})

# from ChatHaruhi.utils import float_array_to_base64

# for text,embed1, embed2 in zip(chunk_data, embed_openai, embed_bge):
#     encode_vec1 = float_array_to_base64(embed1)
#     encode_vec2 = float_array_to_base64(embed2)
#     datas.append({ 'text':text , embed_name:encode_vec1, embed_name_bge:encode_vec2})


In [ ]:
# print(datas[3])

{'text': "narrator:STEWART He's right on target. Height. Weight... She checks Caesar's ears.\nCaesar:「Dairy Queen.」\nWILL:「After. Sit still.」\nSTEWART:「What about diet?」\nWILL:「A healthy diet. Fruits, vegetables.」\nCaesar:「Ice cream. Will ignores him.」\nSTEWART:「What's he saying?」\nWILL:「He's hungry. Caesar's angling to climb off the examining table. Will sends him a look - out it out.」\nSTEWART:「He seems extremely intelligent. How many words does Caesar sign? Beat.」\nWILL:「You know, the basics.」\nCaesar:「Soft-serve.」\n", 'luotuo_openai': 'vHNa1LxkZe08C1U/vHiHHDynVaQ8voAfPAhovbziX7q8EjFcu+VpBj0oHyo9KB8qvHgT9DvvwZY8GhCSu0qIhD0RZ9Y6QC/0PICJh70nrAI6DOhKvDF0obxQJ/S7FTqoOhP9+roBGXg82p1Ou8I2Jrv6jU67cjrxPP+AAztMVSO7sBsqvMEWQ7x/Rm+8iWvWvEe4zL0j9fs7mmbwu7F0oTvNkdC7nFBYPD/2YLwEIsS8eIccuvvmxTzm/BG8glYlu5wzjjteGcI8pPkUPH2zZLzPtMw8I0k/PEM5PzxOPos8y/7FvFJK8LuG1bK8vfAtPC5rVTyGYoq8spSEPEecArwoWL28RALFuohovTwavU27ZWkGPQQixDwr8fs8yUvXu/R6t7wqe7o9FwdGu7Dkr7poVYg76iInN87rRzwwqxu77JuBvD1Dcrwv4ZY9MZFrPJ2NBT

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


!ls /content/drive/MyDrive/Role_from_RoleLLM

Caesar.jsonl


In [ ]:

import json

def write_jsonl(datas, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        for data in datas:
            json_str = json.dumps(data, ensure_ascii=False)
            f.write(json_str+"\n")



In [ ]:

for role_name in sorted_names:
    script_path = get_script_path(role_name)
    data = read_jsonl(script_path)
    chunk_data = divide_data_into_chunk(data)

    embed_openai = get_embeddings_openai(chunk_data[:])
    embed_bge = get_bge_embeddings_safe(chunk_data[:])

    datas = []

    system_prompt = get_system_prompt( role_name )

    embed_name = 'luotuo_openai'
    embed_name_bge = 'bge_en_s15'

    datas.append({ 'text':system_prompt , embed_name:'system_prompt', embed_name_bge:'system_prompt'})
    datas.append({ 'text':'Reserve Config Setting Here' , embed_name:'config', embed_name_bge:'config'})

    from ChatHaruhi.utils import float_array_to_base64

    for text,embed1, embed2 in zip(chunk_data, embed_openai, embed_bge):
        encode_vec1 = float_array_to_base64(embed1)
        encode_vec2 = float_array_to_base64(embed2)
        datas.append({ 'text':text , embed_name:encode_vec1, embed_name_bge:encode_vec2})

    save_name = "/content/drive/MyDrive/Role_from_RoleLLM/" + role_name + ".jsonl"
    write_jsonl(datas,save_name)


  8%|▊         | 1/13 [00:00<00:01,  8.45it/s]WARNING:langchain.embeddings.openai:Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: Internal server error {
    "error": {
        "message": "Internal server error",
        "type": "auth_subrequest_error",
        "param": null,
        "code": "internal_error"
    }
}
 500 {'error': {'message': 'Internal server error', 'type': 'auth_subrequest_error', 'param': None, 'code': 'internal_error'}} {'Date': 'Fri, 20 Oct 2023 04:13:42 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '166', 'Connection': 'keep-alive', 'vary': 'Origin', 'x-request-id': '64d051740109e038c8f31c5bea2aa30c', 'strict-transport-security': 'max-age=15724800; includeSubDomains', 'CF-Cache-Status': 'DYNAMIC', 'Server': 'cloudflare', 'CF-RAY': '818e6de34f787ba4-LAX', 'alt-svc': 'h3=":443"; ma=86400'}.
 23%|██▎       | 3/13 [00:09<00:30,  3.04s/it]WARNING:langchain.embeddings.op

warning!


  4%|▍         | 66/1716 [00:10<03:42,  7.41it/s]

warning!


  4%|▍         | 72/1716 [00:11<04:12,  6.52it/s]

warning!


  5%|▍         | 79/1716 [00:13<04:40,  5.83it/s]

warning!


  6%|▌         | 95/1716 [00:15<03:35,  7.51it/s]

In [ ]:
for role_name in sorted_names:
    save_name = "/content/drive/MyDrive/Role_from_RoleLLM/" + role_name + ".jsonl"
    if os.path.exists(save_name):
        print('skip ',role_name)
        continue
    # continue

    script_path = get_script_path(role_name)
    data = read_jsonl(script_path)
    chunk_data = divide_data_into_chunk(data)

    if len(chunk_data) > 1500:
        chunk_data = chunk_data[:1500]

    embed_openai = get_embeddings_openai(chunk_data[:])
    embed_bge = get_bge_embeddings_safe(chunk_data[:])

    datas = []

    system_prompt = get_system_prompt( role_name )

    embed_name = 'luotuo_openai'
    embed_name_bge = 'bge_en_s15'

    datas.append({ 'text':system_prompt , embed_name:'system_prompt', embed_name_bge:'system_prompt'})
    datas.append({ 'text':'Reserve Config Setting Here' , embed_name:'config', embed_name_bge:'config'})

    from ChatHaruhi.utils import float_array_to_base64

    for text,embed1, embed2 in zip(chunk_data, embed_openai, embed_bge):
        encode_vec1 = float_array_to_base64(embed1)
        encode_vec2 = float_array_to_base64(embed2)
        datas.append({ 'text':text , embed_name:encode_vec1, embed_name_bge:encode_vec2})


    write_jsonl(datas,save_name)


skip  Caesar
skip  Sonny
skip  Angel
skip  Jigsaw
skip  John Doe
skip  Freddy Krueger
skip  Colonel Hans Landa
skip  Gregory House
skip  Gaston
skip  HAL 9000
skip  Mark Renton
skip  Coriolanus
skip  Oliver Queen
skip  Bruno Antony
skip  D_Artagnan
skip  Dr. Frank-N-Furter
skip  Tugg Speedman
skip  Stifler
skip  Jeff Spicoli
skip  Rorschach
skip  Paul Vitti
skip  Logan
skip  Judge Dredd
skip  Karl Childers
skip  Rachel Lang
skip  Queen Elizabeth I
skip  Tyrion Lannister
skip  John Keating
skip  Wade Wilson
skip  Lyn Cassady
skip  Dr. Hannibal Lecter
skip  Violet Weston
skip  Po
skip  Malcolm X
skip  Willie Soke
skip  Jack Torrance
skip  Alvy Singer
skip  Colonel Nathan R. Jessep
skip  Andrew Detmer
skip  Fred Flintstone
skip  Frank T.J. Mackey
skip  Stephen Hawking
skip  Lestat de Lioncourt
skip  Jack Sparrow
skip  John Coffey
skip  Murphy MacManus
skip  John Dillinger
skip  Jackie Moon
skip  Peter Parker
skip  Abraham Lincoln
skip  James Carter
skip  Tyler Hawkins
skip  Stanley Ipkiss

100%|██████████| 47/47 [18:46<00:00, 23.97s/it]


In [ ]:
# embed_bge = get_bge_embeddings_safe(chunk_data[:])

datas = []

system_prompt = get_system_prompt( role_name )

embed_name = 'luotuo_openai'
embed_name_bge = 'bge_en_s15'

datas.append({ 'text':system_prompt , embed_name:'system_prompt', embed_name_bge:'system_prompt'})
datas.append({ 'text':'Reserve Config Setting Here' , embed_name:'config', embed_name_bge:'config'})

from ChatHaruhi.utils import float_array_to_base64

for text,embed1, embed2 in zip(chunk_data, embed_openai, embed_bge):
    encode_vec1 = float_array_to_base64(embed1)
    encode_vec2 = float_array_to_base64(embed2)
    datas.append({ 'text':text , embed_name:encode_vec1, embed_name_bge:encode_vec2})


write_jsonl(datas,save_name)

# 生成github的描述

In [ ]:
desc_path = "/content/RoleBench/profiles-eng/desc.json"
movie_name_path = "/content/RoleBench/profiles-eng/scripts.json"

# 请补全下面的python代码为我解析这两个json文件

import json

with open(desc_path) as f:
    desc = json.load(f)

with open(movie_name_path) as f:
    movie_names = json.load(f)


In [ ]:
print('''为下面的markdown表格的第三列，增加这个角色的中文描述。


例子输入:
Jack Sparrow  |  Pirates-of-the-Caribbean-Dead-Man's-Chest  |

例子输出:
Jack Sparrow  |  Pirates-of-the-Caribbean-Dead-Man's-Chest  | 加勒比海盗中的杰克船长

输入:
''')

print('角色 | 电影 | 中文')

print('---|---|---')

for role_name in movie_names:
    print(role_name , ' | ', movie_names[role_name] , ' | ', )

为下面的markdown表格的第三列，增加这个角色的中文描述。


例子输入:
Jack Sparrow  |  Pirates-of-the-Caribbean-Dead-Man's-Chest  | 

例子输出:
Jack Sparrow  |  Pirates-of-the-Caribbean-Dead-Man's-Chest  | 加勒比海盗中的杰克船长

输入:

角色 | 电影 | 中文
---|---|---
HAL 9000  |  2001-A-Space-Odyssey  | 
Colonel Nathan R. Jessep  |  A-Few-Good-Men  | 
Antonio Salieri  |  Amadeus  | 
Stifler  |  American-Pie  | 
Paul Vitti  |  Analyze-That  | 
Alvy Singer  |  Annie-Hall  | 
Violet Weston  |  August-Osage-County  | 
Willie Soke  |  Bad-Santa  | 
Gaston  |  Beauty-and-the-Beast  | 
The Dude  |  Big-Lebowski,-The  | 
Murphy MacManus  |  Boondock-Saints,-The  | 
Paul Conroy  |  Buried  | 
Truman Capote  |  Capote  | 
Mater  |  Cars-2  | 
Andrew Detmer  |  Chronicle  | 
Coriolanus  |  Coriolanus  | 
Benjamin Button  |  Curious-Case-of-Benjamin-Button,-The  | 
John Keating  |  Dead-Poets-Society  | 
Wade Wilson  |  Deadpool  | 
Jim Morrison  |  Doors,-The  | 
Queen Elizabeth I  |  Elizabeth-The-Golden-Age  | 
Jeff Spicoli  |  Fast-Times-at-Ridg

In [ ]:
ch_desc = """《2001太空漫游》中的HAL 9000电脑
《好汉两三个》中的内森·R·杰瑟普上校
《阿玛迪斯》中的安东尼奥·萨列里
《美国派》中的斯蒂夫勒
《心理分析那件事》中的保罗·维蒂
《安妮·霍尔》中的阿尔维·辛格
《奥塞奇郡的八月》中的紫罗兰·韦斯顿
《坏圣诞老人》中的威利·索克
《美女与野兽》中的加斯顿
《大勒布斯基》中的“大佬”
《天使之城》中的墨菲·麦克马纳斯
《活埋》中的保罗·康罗伊
《卡波特》中的杜鲁门·卡波特
《赛车总动员2》中的玛特
《编年史》中的安德鲁·德特默
《科里奥兰纳斯》中的主角
《本杰明·巴顿奇事》中的本杰明·巴顿
《死亡诗社》中的约翰·基廷
《死侍》中的韦德·威尔逊
《门》中的吉姆·莫里森
《伊丽莎白：黄金时代》中的伊丽莎白一世女王
《瑞奇蒙特高中时光》中的杰夫·斯皮科利
《石头家族》中的弗雷德·弗林斯通
《弗雷迪对杰森》中的弗雷迪·克鲁格
《权力的游戏》中的提利昂·兰尼斯特
《起身舞蹈》中的詹姆斯·布朗
《老无所依》中的沃尔特·科瓦尔斯基
《绿里奇迹》中的约翰·科菲
《她》中的西奥多·特温布利
《豪斯医生》中的格雷戈里·豪斯
《我，机器人》中的桑尼
《无耻混蛋》中的汉斯·兰达上校
《德莱德法官》中的法官德莱德
《朱诺》中的朱诺·麦克夫
《功夫熊猫》中的阿宝
《夫人杀手》中的G.H.多尔教授
《撒谎的男人》中的弗莱彻·里德
《林肯》中的亚伯拉罕·林肯
《木兰花》中的弗兰克 T.J. 麦凯
《马尔科姆X》中的马尔科姆X
《记忆碎片》中的伦纳德·谢尔比
《牛奶》中的哈维·牛奶
《飞越疯人院》中的兰德尔·麦克默菲
《加勒比海盗》中的杰克·斯派洛船长
《公敌》中的约翰·迪林格
《诅咒女王》中的莱斯塔特·德·莱昂科特
《记得我》中的泰勒·霍金斯
《猩球崛起》中的凯撒
《房间》中的杰克
《尖峰时刻2》中的詹姆斯·卡特
《电锯惊魂》中的拼图杀手
《七宗罪》中的约翰·多
《半职业球员》中的杰基·月亮
《夏洛克·福尔摩斯》中的夏洛克·福尔摩斯
《史莱克》中的史莱克
《乌云背后的幸福线》中的帕特·索利塔诺
《刀锯》中的卡尔·柴尔德斯
《蜘蛛侠》中的彼得·帕克
《列车上的陌生人》中的布鲁诺·安东尼
《超级糟糕》中的塞思
《纽约奇缘》中的卡登·科塔德
《出租车司机》中的特拉维斯·比克尔
《面具》中的斯坦利·伊普基斯
《盯羊的男人》中的林恩·卡萨迪
《办公室》中的迈克尔·斯科特
《名望》中的罗伯特·安吉尔
《瑞秋的愤怒：凯丽2》中的瑞秋·朗
《洛奇恐怖秀》中的弗兰克·N·福特医生
《闪灵》中的杰克·托兰斯
《天才雷普利》中的汤姆·雷普利
《三剑客》中的达达尼昂
《万物理论》中的斯蒂芬·霍金
《雷神：诸神黄昏》中的雷神索尔
《明日帝国》中的詹姆斯·邦德
《迷幻列车》中的马克·伦顿
《热带惊雷》中的塔格·斯皮德曼
《香草天空》中的大卫·艾姆斯
《守望者》中的罗夏克
《华尔街之狼》中的乔丹·贝尔福特
《X战警：金刚狼》中的洛根
《疯狂动物城》中的朱迪·胡普斯
《神秘博士》中的博士
《绯闻女孩》中的布莱尔·沃尔多夫
《正当防卫》中的雷兰·吉文斯
《塞勒姆》中的玛丽·西布利
《路西法》中的路西法·晨星
《生活大爆炸》中的谢尔顿·库珀
《我的小马驹：友谊之魔》中的暮光星辉
《绿箭侠》中的奥利弗·皇后
《海军罪案调查处》中的利洛伊·杰斯罗·吉布斯
《天使》中的天使
《始祖家族》中的克劳斯·米卡尔森
《王权》中的凯瑟琳女王
《汉尼拔》中的汉尼拔·莱克特医生
《星期五之光》中的教练埃里克·泰勒"""

ch_descs = ch_desc.split('\n')


In [ ]:
print('角色 | 电影 | 中文 | 字段 ')

print('---|---|---|---')

count = 0

for role_name in movie_names:
    print(role_name , ' | ', movie_names[role_name] , ' | ', ch_descs[count] , '|' , "silk-road/ChatHaruhi-from-RoleLLM/" + role_name )
    count += 1

角色 | 电影 | 中文 | 字段 
---|---|---|---
HAL 9000  |  2001-A-Space-Odyssey  |  《2001太空漫游》中的HAL 9000电脑 | silk-road/ChatHaruhi-from-RoleLLM/HAL 9000
Colonel Nathan R. Jessep  |  A-Few-Good-Men  |  《好汉两三个》中的内森·R·杰瑟普上校 | silk-road/ChatHaruhi-from-RoleLLM/Colonel Nathan R. Jessep
Antonio Salieri  |  Amadeus  |  《阿玛迪斯》中的安东尼奥·萨列里 | silk-road/ChatHaruhi-from-RoleLLM/Antonio Salieri
Stifler  |  American-Pie  |  《美国派》中的斯蒂夫勒 | silk-road/ChatHaruhi-from-RoleLLM/Stifler
Paul Vitti  |  Analyze-That  |  《心理分析那件事》中的保罗·维蒂 | silk-road/ChatHaruhi-from-RoleLLM/Paul Vitti
Alvy Singer  |  Annie-Hall  |  《安妮·霍尔》中的阿尔维·辛格 | silk-road/ChatHaruhi-from-RoleLLM/Alvy Singer
Violet Weston  |  August-Osage-County  |  《奥塞奇郡的八月》中的紫罗兰·韦斯顿 | silk-road/ChatHaruhi-from-RoleLLM/Violet Weston
Willie Soke  |  Bad-Santa  |  《坏圣诞老人》中的威利·索克 | silk-road/ChatHaruhi-from-RoleLLM/Willie Soke
Gaston  |  Beauty-and-the-Beast  |  《美女与野兽》中的加斯顿 | silk-road/ChatHaruhi-from-RoleLLM/Gaston
The Dude  |  Big-Lebowski,-The  |  《大勒布斯基》中的“大佬” | silk-r